# Fisher FIFO

In the `fisher-fifo-v3` notebook, we set a code to use the Fisher information effectively in neural netwrks using the *first-in-first-out* strategy to store gradients.

Here our objective is to assess the effect of the FIFO buffer in the quality of the model. Also, we implement the partitioning strategy to make our algorithm generalizable to larger networks, as well as datasets with larger instances (like images).

To enhance the partition effectiveness, we proceed to use the "maximum-block-update", to make the algorithm faster.

---

in fisher `fisher-fifo-v4` notebook we are trying to implement the algorithm in a more efficient way by using the [torch.bmm](https://pytorch.org/docs/stable/generated/torch.bmm.html). The main idea is to execute the matrix-multiplications of more than one block in an optimized way.

---

in `fisher-fifo-v4.2` notebook, we are trying to make the algorithm even faster using a single centralized object responsible for storing all the matrices (and their inverses) as well as all gradients and buffers. The main idea here is to make the most of vectorization using Pytorch utilities for GPU.

---

in `fisher-fifo-v4.3` notebook, we try to take the algorithm one step further in efficiency. We are implementing the strategy to retrieve the partitions in sets, instead of individually.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import math
import os
import json

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# from skopt import gp_minimize

from scipy import stats

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/fisher-fifo-v4-3/bayes-opt-results.npz
/kaggle/input/fisher-fifo-v4-3/results_step_100.json
/kaggle/input/fisher-fifo-v4-3/__results__.html
/kaggle/input/fisher-fifo-v4-3/__notebook__.ipynb
/kaggle/input/fisher-fifo-v4-3/__output__.json
/kaggle/input/fisher-fifo-v4-3/custom.css
/kaggle/input/fisher-fifo-v4-3/data/MNIST/raw/t10k-labels-idx1-ubyte
/kaggle/input/fisher-fifo-v4-3/data/MNIST/raw/t10k-images-idx3-ubyte.gz
/kaggle/input/fisher-fifo-v4-3/data/MNIST/raw/t10k-labels-idx1-ubyte.gz
/kaggle/input/fisher-fifo-v4-3/data/MNIST/raw/train-labels-idx1-ubyte.gz
/kaggle/input/fisher-fifo-v4-3/data/MNIST/raw/t10k-images-idx3-ubyte
/kaggle/input/fisher-fifo-v4-3/data/MNIST/raw/train-labels-idx1-ubyte
/kaggle/input/fisher-fifo-v4-3/data/MNIST/raw/train-images-idx3-ubyte
/kaggle/input/fisher-fifo-v4-3/data/MNIST/raw/train-images-idx3-ubyte.gz


In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

import torchvision
import torchvision.datasets as datasets

In [4]:
def get_device():    
    if torch.cuda.is_available():

        device = torch.device('cuda')
        print( torch.cuda.get_device_name(device) )
        print( torch.cuda.get_device_properties(device) )

    else:
        device = torch.device('cpu')
        print(device)
        
    return device

In [5]:
!pip install torchsummary
import torchsummary

In [6]:
class cfg:
    # n_features = 28 * 28
    img_size = (32, 32)
    img_channels = 3
    
    n_samples = 10000
    n_classes = 10  ## we have 10 classes in MNIST
    
    # device = torch.device('cpu')
    device = get_device()
    
    max_loss = 20.0

Tesla P100-PCIE-16GB
_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


# create the dataset

In [7]:
def generate_dataset_mnist(batch_size):
    print(f'generating MNIST data with {cfg.n_classes} classes')
    
    transf_ = torchvision.transforms.Compose([
        # torchvision.transforms.Resize(size=[14, 14]),
        torchvision.transforms.ToTensor()
    ])
    
    mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transf_)
    mnist_test  = datasets.MNIST(root='./data', train=False, download=True, transform=transf_)
    
    mnist_train_dataloader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True)
    mnist_test_dataloader  = DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False)

    return mnist_train_dataloader, mnist_test_dataloader

In [8]:
def generate_dataset_cifar10(batch_size):
    print(f'generating CIFAR10 data with {cfg.n_classes} classes')
    
    transf_ = torchvision.transforms.Compose([
        # torchvision.transforms.Resize(size=[14, 14]),
        torchvision.transforms.ToTensor()
    ])
    
    cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transf_)
    cifar10_test  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transf_)
    
    cifar10_train_dataloader = DataLoader(dataset=cifar10_train, batch_size=batch_size, shuffle=True)
    cifar10_test_dataloader  = DataLoader(dataset=cifar10_test, batch_size=batch_size, shuffle=False)

    return cifar10_train_dataloader, cifar10_test_dataloader

## declaring network architecture

In [9]:
def get_default_network(c=16, device=cfg.device):
    net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=cfg.n_features, out_features=c),
        nn.ReLU(),
        nn.Linear(in_features=c, out_features=c),
        nn.ReLU(),
        nn.Linear(in_features=c, out_features=c),
        nn.ReLU(),
        nn.Linear(in_features=c, out_features=cfg.n_classes)
    )
    
    torchsummary.summary(net, input_size=[[cfg.n_features]], device='cpu')
    
    return net

In [10]:
def get_cnn_network(in_channels=cfg.img_channels, c=16, p_drop=0.1, device=cfg.device):
    
    img_flat_size = (4 * c * (cfg.img_size[0] // 8) * (cfg.img_size[1] // 8) )
    print(img_flat_size)
    net = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=c, kernel_size=5, stride=2, padding=2),
        nn.ReLU(),

        nn.Conv2d(in_channels=c, out_channels=(2 * c), kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout2d(p=p_drop),
        
        nn.Conv2d(in_channels=(2 * c), out_channels=(4 * c), kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout2d(p=p_drop),
        
        nn.Flatten(),
        
        nn.Linear(in_features=img_flat_size, out_features=(8 * c) ),
        nn.ReLU(),
        nn.Dropout(p=p_drop),
        
        nn.Linear(in_features=(8 * c), out_features=(4 * c) ),
        nn.ReLU(),
        nn.Dropout(p=p_drop),
        
        nn.Linear(in_features=(4 * c), out_features=cfg.n_classes)
    )
    
    torchsummary.summary(net, input_size=[[cfg.img_channels, *cfg.img_size]], device='cpu')
    
    return net

In [11]:
def get_cnn_network_v2(in_channels=cfg.img_channels, p_drop=0.1, device=cfg.device):
    
    net = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=96, kernel_size=5, padding=2),
        nn.MaxPool2d(kernel_size=2),
        nn.ReLU(),

        nn.Conv2d(in_channels=96, out_channels=80, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout2d(p=p_drop),
        
        nn.Conv2d(in_channels=80, out_channels=96, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.Dropout2d(p=p_drop),
        
        nn.Conv2d(in_channels=96, out_channels=64, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.Dropout2d(p=p_drop),
        
        nn.Flatten(),
        
        nn.Linear(in_features=4096, out_features=256 ),
        nn.ReLU(),
        nn.Dropout(p=p_drop),
        
        nn.Linear(in_features=256, out_features=cfg.n_classes)
    )
    
    torchsummary.summary(net, input_size=[[cfg.img_channels, *cfg.img_size]], device='cpu')
    
    return net

# object for calculation of the metrics

In [12]:
class Metrics():
    def __init__(self, value_round=None, time_round=None):
        self.metrics_dict = {}
        self.set_initial_time()
        self.val_round = value_round
        self.time_round = time_round
        
    def set_initial_time(self):
        self.init_time = time.time()
        
    def get_time(self):
        return time.time() - self.init_time
    
    def add(self, key, value, step=None):
        
        if step is None:
            step = np.nan
        
        if key not in self.metrics_dict:
            self.metrics_dict[key] = []
        
        t = self.get_time()
        if self.time_round is not None:
            t = round(t, ndigits=self.time_round)
        
        if self.val_round is not None:
            value = round(value, ndigits=self.val_round)
        
        self.metrics_dict[key].append( (value, step, t) )
    
    def add_(self, dict_, step=None):
        for key, value in dict_.items():
            self.add(key, value, step)
    
    def get(self, key, get_step=False, get_time=False):
        y, x, t = zip(*self.metrics_dict[key])
        y, x, t = list(y), list(x), list(t)
        
        return x, y, t

# Fisher Information calculation objects

In [13]:
class FisherFIFO():
    def __init__(self,
                 named_params,
                 buffer_size,
                 partition_size,
                 block_updates):
        
        self.buffer_size = buffer_size
        self.partition_size = partition_size
        self.block_updates = block_updates
        
        named_params = list(named_params)
        
        self.partition_fisher_list = []
        total_partitions, total_block_upd = 0, 0
        for pi, (n, p) in enumerate( named_params ):
            part_fisher = PartitionerFisherFIFO(param = p,
                                                name = n,
                                                buffer_size = buffer_size,
                                                partition_size = partition_size,
                                                block_updates = block_updates,
                                                parent_fifo = None)
            
            self.partition_fisher_list.append( (p, part_fisher, total_partitions, total_block_upd) )
            
            total_partitions += part_fisher.num_part
            total_block_upd += part_fisher.block_updates
            
        self.num_part = total_partitions
        self.total_block_updates = total_block_upd
        
        print(f'total partitions: {self.num_part} - effective block updates: {self.total_block_updates}')
        
        ## pre-alocate the memory for the tensor that stores the selected gradients (changes every iteration)
        self.g = torch.zeros(size=[self.total_block_updates, partition_size, 1], dtype=torch.float, device=cfg.device)
        
        ## pre-alocate the memory for the tensor that stores the buffer and the tensor for the inverse
        self.buffer = torch.zeros(size=[self.num_part, partition_size, buffer_size], dtype=torch.float, device=cfg.device)
        self.fisher_inv = torch.zeros(size=[self.num_part, partition_size, partition_size], dtype=torch.float, device=cfg.device)        
    
        print('initializing buffers and inverses...')
        ## now we initialize the buffer and the inverse for all partitions
        i = 0
        for _, part_fisher, _, _ in self.partition_fisher_list:
            for _, start, end in part_fisher.ind_fisher_list:

                if i == 0 or ( (i + 1) % 10000 ) == 0 or i == (self.num_part - 1):
                    print(f'partition {i+1}/{self.num_part}')

                n = end - start
                buffer, _, fisher_inv = self.initialize_fisher_partition(param_size=n, buffer_size=self.buffer_size)

                self.buffer[i, :n, :] = buffer
                self.fisher_inv[i, :n, :n] = fisher_inv

                i += 1

    
    def initialize_fisher_partition(self, param_size, buffer_size):
        
        buffer = self.get_initial_buffer_v2(param_size, buffer_size)
            
        ## shuffle buffer across columns
        buffer = buffer[:, torch.randperm(buffer.shape[1]) ]
        
        ## the fisher matrix will be initialized as G @ G.T, in which G is our buffer. We built
        ## our buffer in a smart way so the resulting Fisher info matrix is initialized close to identity
        fisher = buffer @ buffer.T
        
        ## since our Fisher information is diagonal for now, its inverse is given just by the innverted
        ## elements of the diagonal. 
        fisher_inv = torch.diag( 1 / torch.diag(fisher) )
        
        return buffer, fisher, fisher_inv
    
    
    def get_initial_buffer_v2(self, param_size, buffer_size):
        ## here we adopt a faster approach to initialize the buffer. We use n 
        ## identity matrices concatenated column-wise. n is determined by `param_size` and `buffer_size`
        
        n_eye = math.ceil(buffer_size / param_size)
        I = torch.eye(n=param_size, dtype=torch.float, device=cfg.device)
        
        buffer = torch.cat(n_eye * [I], dim=1)[:, :buffer_size]
        
        assert buffer.shape == (param_size, buffer_size)
        
        return buffer


    def get_idx_lists(self):
        run_enc_list = []
        default_idx_list = []
        for p, part_fisher, num_part, block_upd in self.partition_fisher_list:
            init_block, end_block, g_init_idx, g_end_idx = part_fisher.get_random_blocks()
            
            # print(f'param shape: {p.shape} - blocks: {init_block} to {end_block} - grad: {g_init_idx} to {g_end_idx}')
            
            run_enc_list.append( (num_part + init_block, num_part + end_block, g_init_idx, g_end_idx) )
            default_idx_list.append( np.arange(start=num_part + init_block, stop=num_part + end_block + 1) )
            
            
        return run_enc_list, np.concatenate(default_idx_list)
    
    
    def read_gradients(self, idx):
        self_g_start = 0
        for i, (_, _, g_start, g_end) in enumerate(idx):
            n_grad = g_end - g_start
            # self_g_end = min( self_g_start + n_grad, torch.numel(self.g) )
            self_g_end = self_g_start + n_grad
            
            p, _, _, _ = self.partition_fisher_list[i]
            
            self.g.view(-1)[self_g_start:self_g_end] = p.grad.view(-1)[g_start:g_end]
            
            if (n_grad % self.partition_size) > 0:
                extra_zeros = self.partition_size - (n_grad % self.partition_size)
                self.g.view(-1)[self_g_end:(self_g_end + extra_zeros)] = 0.0
            else:
                extra_zeros = 0
            
#             print(f'self_g_start: {self_g_start} - self_g_end: {self_g_end} - self.g.shape: {self.g.view(-1).shape}')
#             print(f'g_start: {g_start} - g_end: {g_end} - p.grad.shape: {p.grad.view(-1).shape}')
#             print(f'n_grad: {n_grad} - part-size: {self.partition_size} - extra-zeros: {extra_zeros}')
#             print()
            
            self_g_start = self_g_end + extra_zeros
            

    def write_gradients(self, idx):
        self_g_start = 0
        for i, (_, _, g_start, g_end) in enumerate(idx):
            n_grad = g_end - g_start
            self_g_end = self_g_start + n_grad
            
            p, _, _, _ = self.partition_fisher_list[i]
            p.grad.view(-1)[g_start:g_end] = self.g.view(-1)[self_g_start:self_g_end]

            if (n_grad % self.partition_size) > 0:
                extra_zeros = self.partition_size - (n_grad % self.partition_size)
            else:
                extra_zeros = 0
            
            self_g_start = self_g_end + extra_zeros

    
    def step(self):
        ## selects the blocks to be updated
        run_enc_idx, default_idx = self.get_idx_lists()
        
        ## read the selected blocks gradients and stores them in self.g
        self.read_gradients(run_enc_idx)
        
        ## get apart the inverses and buffers for the selected blocks
        inv = self.fisher_inv[default_idx, ...]
        buffer = self.buffer[default_idx, ...]
        
        # print(inv.shape, buffer.shape)
        
        ## update the buffer
        
        ## dimensions are: partitions, gradient-size, bufffer-size
        g_old = buffer[:, :, 0:1] 
        
        ## we  update in the third dimension: "buffer-size"
        buffer = torch.cat([buffer[:, :, 1:], self.g], dim=2)
        
        ## update the inverses and modify current gradients
        ## ...
        sqrt_NB = math.sqrt(self.buffer_size)
        
        ## update inverse - phase 1: add new gradient ##
        g_phase1 = self.upd_inverse( (1 / sqrt_NB) * self.g, inv, type_='sum')

        ## update inverse - phase 2: remove old gradient ##
        self.upd_inverse( (1 / sqrt_NB) * g_old, inv, type_='sub')

        ## modify the current gradients
        if False:
            ## use the "phase-1-trick" to get the estimated new gradient
            self.g = g_phase1 * sqrt_NB
        else:
            ## get the modified gradient using "de facto" the new inverses and the gradients
            self.g = self.modify_grad(self.g, inv)
        
        ## return the inverses and buffers to the main tensor
        self.fisher_inv[default_idx, ...] = inv
        self.buffer[default_idx, ...] = buffer
        
        ## return the modified gradients to the parameters
        self.write_gradients(run_enc_idx)


    def upd_inverse(self, g, inverse, type_='sum'):
        ## update the inverse based on the woodbury inversion
        f_inv_g = torch.bmm(inverse, g)

        if type_ == 'sum':
            d = 1 + torch.sum(g * f_inv_g, dim=[1, 2], keepdim=True)
            inverse[:] = inverse - (f_inv_g * torch.transpose(f_inv_g, 1, 2) / d)

        elif type_ == 'sub':
            d = 1 - torch.sum(g * f_inv_g, dim=[1, 2], keepdim=True)
            inverse[:] = inverse + (f_inv_g * torch.transpose(f_inv_g, 1, 2) / d)

        else:
            ## incorrect type
            print('incorrect rank-1 update type: ' + type_)
        
        return f_inv_g


    def modify_grad(self, g, inverse):
        return torch.bmm(inverse, g)

In [14]:
class PartitionerFisherFIFO():
    def __init__(self,
                 param,
                 name,
                 buffer_size,
                 partition_size,
                 block_updates,
                 parent_fifo):
        
        self.param = param
        self.name = name 
        
        if partition_size is None:
            self.partition_size = param.numel()
        else:
            self.partition_size = partition_size
        
        ## calculates the number of partitions required. It is calculated using the param size and
        ## our partition maximum size. The gradient (the same size as param) is going to be partitioned in
        ## equal pieces (except possibly the last one) to be processed individually by our "IndividualFisherFIFO"
        self.param_size = param.numel()
        self.num_part = math.ceil(self.param_size / self.partition_size)
        
        ## the number of blocks (partitions) to update at each iteration. This can be < num_part to make
        ## the algorithm more efficient. (we dont update every partition at every iteration)
        if block_updates is None:
            self.block_updates = self.num_part
        else:
            self.block_updates = min(block_updates, self.num_part)
        
        print(f'FisherPartitioner: param: {self.param_size} - partition: {self.partition_size} - nº part: {self.num_part} - block updates: {self.block_updates}')
                
        ## the list stores the indexes used to partition the gradient
        self.ind_fisher_list = []
        for i in range(self.num_part):
            start = i * self.partition_size
            end = min(start + self.partition_size, self.param_size)
            
            self.ind_fisher_list.append( (i, start, end) )
        
    
    def get_random_blocks(self, num_part=None, block_upd=None):
        
        if num_part is None:
            num_part = self.num_part
        
        if block_upd is None:
            block_upd = self.block_updates
        
        ## choose the initial block randomly
        init_block = np.random.choice(num_part - block_upd + 1)
        
        ## the final block will be necessarily `block_upd` blocks further. This means we select
        ## a contiguous sequence of blocks. This is going to be used for performance reasons
        end_block = init_block + block_upd - 1
        
        ## therefore, the starting and ending index to be used to fetch the gradient positions for the
        ## blocks will be the starting index for the first block and the ending positions for the last block
        _, g_init_idx, _ = self.ind_fisher_list[init_block]
        _, _, g_end_idx = self.ind_fisher_list[end_block]
        
        return init_block, end_block, g_init_idx, g_end_idx

---

# utils function for training

In [18]:
def accuracy_score_tns(y_true, y_pred):
    return torch.mean( (y_true == y_pred).to(dtype=torch.float) ).cpu().item()

In [19]:
def train_iteration(x, y, net, optim, loss, fisher=None):
    net.train()
    net.zero_grad()
    
    y_pred = net(x)
    l = loss(y_pred, y)
    
    l.backward()
    
    if fisher is not None:
        fisher.step()
    
    optim.step()
    
    return l.item(), accuracy_score_tns( y.view(-1), y_pred.argmax(dim=1).view(-1) )

In [20]:
def evaluate(net, dataloader, loss):
    net.eval()
    
    with torch.no_grad():

        loss_list = []
        y_pred_list = []
        y_label_list = []
        for x, y in dataloader:
            
            x = x.to(cfg.device)
            y = y.to(cfg.device)

            y_pred = net(x)
            l = loss(y_pred, y)

            loss_list.append( l.cpu().item() )
            y_pred_list.append( y_pred.argmax(dim=1).view(-1) )
            y_label_list.append( y.view(-1) )

        y_pred_list = torch.cat(y_pred_list).view(-1)
        y_label_list = torch.cat(y_label_list).view(-1)

    return np.mean(loss_list), accuracy_score_tns(y_label_list, y_pred_list)

# training

In [21]:
def train_network_fisher_optimization(batch_size = 32,
                                      lr = 1e-3,
                                      momentum = 0.9,
                                      epochs = 30,
                                      buffer_size = 1000,
                                      partition_size = 256,
                                      block_updates = 4,
                                      net_params = {'c':16, 'p':0.1},
                                      apply_fisher = True,
                                      # gpu_memory_check = 20,
                                      time_limit_secs = 600,
                                      interval_print = 100):

    ## declare (instantiate) the dataset
    train_dataloader, test_dataloader = generate_dataset_cifar10(batch_size = batch_size)

    ## instantiate the network
    net = get_cnn_network_v2(p_drop = net_params['p']).to(device=cfg.device)
    
    if apply_fisher:
        ## instantiate FisherFIFO object to create and update the Fisher info matrix
        fisher_fifo = FisherFIFO(named_params = net.named_parameters(),
                                 buffer_size = buffer_size,
                                 partition_size = partition_size,
                                 block_updates = block_updates)
    else:
        fisher_fifo = None

    ## create loss object: we multiply by our constant to stabilize norms
    # cross_entropy = nn.CrossEntropyLoss(reduction='mean') # standard version
    cross_entropy_standard = nn.CrossEntropyLoss(reduction='mean')
    cross_entropy = lambda y_pred, y: math.sqrt(batch_size) * cross_entropy_standard(y_pred, y)
    
    ## create optimize objects
    optim = torch.optim.SGD(params=net.parameters(), lr=lr, momentum=momentum)

    default_metrics = Metrics(value_round=3, time_round=2)

    ini_time = time.time()

    step = 0
    training_finished = False
    for epc in range(1, epochs + 1):
        
        if training_finished:
            break
        
        print(f'starting epoch: {epc}/{epochs}')

        for nbt, (x, y) in enumerate(train_dataloader):

            if training_finished:
                break

            x = x.to(cfg.device)
            y = y.to(cfg.device)

            train_loss, train_acc = train_iteration(x, y, net, optim, cross_entropy, fisher_fifo)
            default_metrics.add_({'train-loss': train_loss, 'train-acc': train_acc}, step=step)
            
            ## check time limit
            t = int(time.time() - ini_time)
            if t > time_limit_secs:
                print('time is up! finishing training')
                training_finished = True

            if ( (nbt + 1) % interval_print ) == 0 or (nbt + 1) == len(train_dataloader) or training_finished:
                avg_train_loss = np.mean( default_metrics.get('train-loss')[1][-interval_print:] )
                avg_train_acc = np.mean( default_metrics.get('train-acc')[1][-interval_print:] )
                
                test_loss, test_acc = evaluate(net, test_dataloader, cross_entropy)
                default_metrics.add_({'test-loss': test_loss, 'test-acc': test_acc}, step=step)

                m, s = t // 60, t % 60

                print(f'batch: {nbt + 1}/{len(train_dataloader)}', end='')
                print(f' - train loss: {avg_train_loss:.4f} - test loss: {test_loss:.4f}', end='')
                print(f' - train acc: {avg_train_acc:.4f} - test acc: {test_acc:.4f}', end='')
                print(f' - {m}m {s}s')
                
            step += 1

        ## check for GPU memory consumption
        if torch.cuda.is_available():
            mem_alloc_gb = torch.cuda.memory_allocated(cfg.device) / 1024**3
            mem_res_gb = torch.cuda.memory_reserved(cfg.device) / 1024**3
            max_mem_alloc_gb = torch.cuda.max_memory_allocated(cfg.device) / 1024**3
            max_mem_res_gb = torch.cuda.max_memory_reserved(cfg.device) / 1024**3

            print(f'GPU memory used: {mem_alloc_gb:.2f} GB - max: {max_mem_alloc_gb:.2f} GB - memory reserved: {mem_res_gb:.2f} GB - max: {max_mem_res_gb:.2f} GB')

            # torch.cuda.empty_cache()

    return default_metrics, fisher_fifo

In [22]:
last_step_saved = None

def results_list_to_json(results_list, out_dir='/kaggle/working', step=0):
    global last_step_saved

    json_results = []

    for metrics, bs, ps, bu in results_list:
        json_results.append({
            'buffer-size': bs,
            'partition-size': ps,
            'blocks-updates': bu,
            'metrics': metrics.metrics_dict
        })

    with open( os.path.join(out_dir, f'results_step_{step}.json'), 'w' ) as fp:
        json.dump(json_results, fp)
    
    if last_step_saved is not None:
        old_file = os.path.join(out_dir, f'results_step_{last_step_saved}.json')
        if os.path.exists(old_file):
            os.remove(old_file)
    
    last_step_saved = step

In [23]:
def get_min_test_loss(metrics):
    _, test_loss, _ = metrics.get('test-loss')
    return min(test_loss)

## executing runs - FisherFIFO

In [24]:
buffer_size = 295
partition_size = 185
block_updates = 85

n_runs = 20

In [25]:
results_list = []
step_i = 0

for _ in range(n_runs):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    print(f'iteration: {step_i} - testing - buf size: {buffer_size} - part size: {partition_size} - block upd: {block_updates}')

    default_metrics, _ = train_network_fisher_optimization(apply_fisher = True,
                                                           buffer_size = buffer_size,
                                                           partition_size = partition_size,
                                                           block_updates = block_updates,
                                                           net_params = {'p': 0.1},
                                                           epochs = 100,
                                                           time_limit_secs = 1200)

    results_list.append( (default_metrics, buffer_size, partition_size, block_updates) )
    results_list_to_json(results_list, step=step_i)
    step_i += 1
    
    print()

iteration: 0 - testing - buf size: 295 - part size: 185 - block upd: 85
generating CIFAR10 data with 10 classes


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 32, 32]           7,296
         MaxPool2d-2           [-1, 96, 16, 16]               0
              ReLU-3           [-1, 96, 16, 16]               0
            Conv2d-4           [-1, 80, 16, 16]         192,080
              ReLU-5           [-1, 80, 16, 16]               0
         MaxPool2d-6             [-1, 80, 8, 8]               0
         Dropout2d-7             [-1, 80, 8, 8]               0
            Conv2d-8             [-1, 96, 8, 8]         192,096
              ReLU-9             [-1, 96, 8, 8]               0
        Dropout2d-10             [-1, 96, 8, 8]               0
           Conv2d-11             [-1, 64, 8, 8]         153,664
             ReLU-12             [-1, 64, 8, 8]               0
        Dropou

batch: 100/1563 - train loss: 5.2975 - test loss: 5.1798 - train acc: 0.6690 - test acc: 0.6773 - 2m 44s
batch: 200/1563 - train loss: 5.3575 - test loss: 5.2123 - train acc: 0.6638 - test acc: 0.6768 - 2m 48s
batch: 300/1563 - train loss: 5.2669 - test loss: 5.4024 - train acc: 0.6638 - test acc: 0.6624 - 2m 51s
batch: 400/1563 - train loss: 5.4236 - test loss: 5.4470 - train acc: 0.6634 - test acc: 0.6520 - 2m 54s
batch: 500/1563 - train loss: 5.1611 - test loss: 5.1458 - train acc: 0.6798 - test acc: 0.6767 - 2m 58s
batch: 600/1563 - train loss: 5.0754 - test loss: 5.4493 - train acc: 0.6731 - test acc: 0.6669 - 3m 1s
batch: 700/1563 - train loss: 5.1895 - test loss: 5.0614 - train acc: 0.6791 - test acc: 0.6851 - 3m 5s
batch: 800/1563 - train loss: 5.1016 - test loss: 5.1790 - train acc: 0.6732 - test acc: 0.6711 - 3m 8s
batch: 900/1563 - train loss: 5.1253 - test loss: 4.9657 - train acc: 0.6766 - test acc: 0.6863 - 3m 11s
batch: 1000/1563 - train loss: 4.9500 - test loss: 4.9738 

batch: 1100/1563 - train loss: 3.3095 - test loss: 3.7346 - train acc: 0.7943 - test acc: 0.7749 - 6m 49s
batch: 1200/1563 - train loss: 3.2930 - test loss: 3.9233 - train acc: 0.7924 - test acc: 0.7662 - 6m 53s
batch: 1300/1563 - train loss: 3.2005 - test loss: 3.7297 - train acc: 0.8018 - test acc: 0.7718 - 6m 56s
batch: 1400/1563 - train loss: 3.1447 - test loss: 3.7143 - train acc: 0.8003 - test acc: 0.7741 - 6m 59s
batch: 1500/1563 - train loss: 3.4344 - test loss: 3.7541 - train acc: 0.7887 - test acc: 0.7689 - 7m 2s
batch: 1563/1563 - train loss: 3.5421 - test loss: 3.7090 - train acc: 0.7803 - test acc: 0.7732 - 7m 5s
GPU memory used: 2.88 GB - max: 3.19 GB - memory reserved: 3.28 GB - max: 3.28 GB
starting epoch: 9/100
batch: 100/1563 - train loss: 2.8636 - test loss: 3.6810 - train acc: 0.8209 - test acc: 0.7740 - 7m 8s
batch: 200/1563 - train loss: 2.9818 - test loss: 3.6133 - train acc: 0.8134 - test acc: 0.7813 - 7m 12s
batch: 300/1563 - train loss: 2.8144 - test loss: 3.6

...

partition 8635/8635
starting epoch: 1/100
batch: 100/1563 - train loss: 13.0237 - test loss: 13.0134 - train acc: 0.0952 - test acc: 0.1065 - 0m 1s
batch: 200/1563 - train loss: 12.9829 - test loss: 12.8431 - train acc: 0.1137 - test acc: 0.1563 - 0m 4s
batch: 300/1563 - train loss: 12.3913 - test loss: 11.6943 - train acc: 0.1832 - test acc: 0.2220 - 0m 7s
batch: 400/1563 - train loss: 11.7433 - test loss: 11.2559 - train acc: 0.2312 - test acc: 0.2627 - 0m 11s
batch: 500/1563 - train loss: 11.3168 - test loss: 11.0816 - train acc: 0.2509 - test acc: 0.2691 - 0m 14s
batch: 600/1563 - train loss: 10.9238 - test loss: 10.4741 - train acc: 0.2781 - test acc: 0.3070 - 0m 17s
batch: 700/1563 - train loss: 10.2950 - test loss: 9.8416 - train acc: 0.3299 - test acc: 0.3520 - 0m 21s
batch: 800/1563 - train loss: 10.0326 - test loss: 9.5958 - train acc: 0.3387 - test acc: 0.3675 - 0m 24s
batch: 900/1563 - train loss: 9.8457 - test loss: 9.4631 - train acc: 0.3377 - test acc: 0.3719 - 0m 27s
ba

batch: 1100/1563 - train loss: 4.2570 - test loss: 4.4370 - train acc: 0.7297 - test acc: 0.7285 - 4m 6s
batch: 1200/1563 - train loss: 4.3098 - test loss: 4.6517 - train acc: 0.7369 - test acc: 0.7116 - 4m 9s
batch: 1300/1563 - train loss: 4.3796 - test loss: 4.3195 - train acc: 0.7272 - test acc: 0.7327 - 4m 13s
batch: 1400/1563 - train loss: 4.0907 - test loss: 4.4421 - train acc: 0.7403 - test acc: 0.7235 - 4m 16s
batch: 1500/1563 - train loss: 4.1841 - test loss: 4.2028 - train acc: 0.7416 - test acc: 0.7423 - 4m 19s
batch: 1563/1563 - train loss: 4.0495 - test loss: 4.3845 - train acc: 0.7518 - test acc: 0.7309 - 4m 22s
GPU memory used: 2.88 GB - max: 3.19 GB - memory reserved: 3.28 GB - max: 3.28 GB
starting epoch: 6/100
batch: 100/1563 - train loss: 4.0321 - test loss: 4.2740 - train acc: 0.7550 - test acc: 0.7348 - 4m 26s
batch: 200/1563 - train loss: 4.0845 - test loss: 4.3300 - train acc: 0.7478 - test acc: 0.7309 - 4m 29s
batch: 300/1563 - train loss: 3.9605 - test loss: 4.

batch: 400/1563 - train loss: 2.5318 - test loss: 3.6155 - train acc: 0.8462 - test acc: 0.7879 - 8m 9s
batch: 500/1563 - train loss: 2.6145 - test loss: 3.5994 - train acc: 0.8356 - test acc: 0.7857 - 8m 13s
batch: 600/1563 - train loss: 2.4966 - test loss: 3.4781 - train acc: 0.8394 - test acc: 0.7900 - 8m 16s
batch: 700/1563 - train loss: 2.7296 - test loss: 3.6640 - train acc: 0.8343 - test acc: 0.7819 - 8m 19s
batch: 800/1563 - train loss: 2.7197 - test loss: 3.6862 - train acc: 0.8372 - test acc: 0.7839 - 8m 22s
batch: 900/1563 - train loss: 2.6611 - test loss: 3.4975 - train acc: 0.8362 - test acc: 0.7931 - 8m 26s
batch: 1000/1563 - train loss: 2.6785 - test loss: 3.6194 - train acc: 0.8381 - test acc: 0.7855 - 8m 30s
batch: 1100/1563 - train loss: 2.6373 - test loss: 3.4404 - train acc: 0.8378 - test acc: 0.7985 - 8m 33s
batch: 1200/1563 - train loss: 2.6160 - test loss: 3.5086 - train acc: 0.8399 - test acc: 0.7917 - 8m 37s
batch: 1300/1563 - train loss: 2.6350 - test loss: 3.

batch: 1400/1563 - train loss: 1.8378 - test loss: 3.6726 - train acc: 0.8863 - test acc: 0.8030 - 12m 18s
batch: 1500/1563 - train loss: 1.8120 - test loss: 3.6858 - train acc: 0.8882 - test acc: 0.7989 - 12m 22s
batch: 1563/1563 - train loss: 1.9051 - test loss: 3.5942 - train acc: 0.8832 - test acc: 0.7989 - 12m 24s
GPU memory used: 2.88 GB - max: 3.19 GB - memory reserved: 3.28 GB - max: 3.28 GB
starting epoch: 15/100
batch: 100/1563 - train loss: 1.3438 - test loss: 3.7136 - train acc: 0.9182 - test acc: 0.8018 - 12m 28s
batch: 200/1563 - train loss: 1.3275 - test loss: 3.7488 - train acc: 0.9186 - test acc: 0.8010 - 12m 31s
batch: 300/1563 - train loss: 1.5650 - test loss: 3.6900 - train acc: 0.8998 - test acc: 0.7998 - 12m 35s
batch: 400/1563 - train loss: 1.4612 - test loss: 3.7870 - train acc: 0.9089 - test acc: 0.7992 - 12m 38s
batch: 500/1563 - train loss: 1.5835 - test loss: 3.7549 - train acc: 0.9045 - test acc: 0.7985 - 12m 41s
batch: 600/1563 - train loss: 1.6073 - test 

batch: 700/1563 - train loss: 1.1285 - test loss: 4.1754 - train acc: 0.9298 - test acc: 0.7961 - 16m 25s
batch: 800/1563 - train loss: 1.0566 - test loss: 4.1813 - train acc: 0.9295 - test acc: 0.8010 - 16m 29s
batch: 900/1563 - train loss: 1.1534 - test loss: 4.0279 - train acc: 0.9317 - test acc: 0.8003 - 16m 32s
batch: 1000/1563 - train loss: 1.0058 - test loss: 4.0923 - train acc: 0.9361 - test acc: 0.7997 - 16m 36s
batch: 1100/1563 - train loss: 1.0983 - test loss: 3.9368 - train acc: 0.9329 - test acc: 0.8081 - 16m 39s
batch: 1200/1563 - train loss: 0.9907 - test loss: 4.2521 - train acc: 0.9355 - test acc: 0.8000 - 16m 43s
batch: 1300/1563 - train loss: 1.1691 - test loss: 4.1882 - train acc: 0.9267 - test acc: 0.8054 - 16m 46s
batch: 1400/1563 - train loss: 1.1054 - test loss: 4.3200 - train acc: 0.9277 - test acc: 0.7993 - 16m 49s
batch: 1500/1563 - train loss: 1.1123 - test loss: 4.0235 - train acc: 0.9289 - test acc: 0.8042 - 16m 53s
batch: 1563/1563 - train loss: 1.0795 - 

In [26]:
for _ in range(n_runs):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    print(f'iteration: {step_i}')

    default_metrics, _ = train_network_fisher_optimization(apply_fisher = False,
                                                           net_params = {'p': 0.1},
                                                           epochs = 100,
                                                           time_limit_secs = 1200)

    results_list.append( (default_metrics, -1, -1, -1) )
    results_list_to_json(results_list, step=step_i)
    step_i += 1
    
    print()

iteration: 20
generating CIFAR10 data with 10 classes
Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 32, 32]           7,296
         MaxPool2d-2           [-1, 96, 16, 16]               0
              ReLU-3           [-1, 96, 16, 16]               0
            Conv2d-4           [-1, 80, 16, 16]         192,080
              ReLU-5           [-1, 80, 16, 16]               0
         MaxPool2d-6             [-1, 80, 8, 8]               0
         Dropout2d-7             [-1, 80, 8, 8]               0
            Conv2d-8             [-1, 96, 8, 8]         192,096
              ReLU-9             [-1, 96, 8, 8]               0
        Dropout2d-10             [-1, 96, 8, 8]               0
           Conv2d-11             [-1, 64, 8, 8]         153,664
             ReLU-12             [-1,

batch: 1000/1563 - train loss: 5.4929 - test loss: 5.3985 - train acc: 0.6513 - test acc: 0.6630 - 2m 39s
batch: 1100/1563 - train loss: 5.4734 - test loss: 5.2466 - train acc: 0.6531 - test acc: 0.6779 - 2m 42s
batch: 1200/1563 - train loss: 5.3955 - test loss: 5.3796 - train acc: 0.6647 - test acc: 0.6625 - 2m 45s
batch: 1300/1563 - train loss: 5.2556 - test loss: 5.1176 - train acc: 0.6713 - test acc: 0.6819 - 2m 47s
batch: 1400/1563 - train loss: 5.3796 - test loss: 5.4091 - train acc: 0.6632 - test acc: 0.6624 - 2m 50s
batch: 1500/1563 - train loss: 5.4044 - test loss: 5.1970 - train acc: 0.6675 - test acc: 0.6790 - 2m 53s
batch: 1563/1563 - train loss: 5.5181 - test loss: 5.0861 - train acc: 0.6503 - test acc: 0.6860 - 2m 55s
GPU memory used: 0.02 GB - max: 3.19 GB - memory reserved: 0.15 GB - max: 3.28 GB
starting epoch: 5/100
batch: 100/1563 - train loss: 4.7349 - test loss: 4.9716 - train acc: 0.7016 - test acc: 0.6931 - 2m 58s
batch: 200/1563 - train loss: 4.9515 - test loss:

batch: 300/1563 - train loss: 3.1024 - test loss: 4.1526 - train acc: 0.8068 - test acc: 0.7548 - 6m 3s
batch: 400/1563 - train loss: 2.9378 - test loss: 4.2616 - train acc: 0.8268 - test acc: 0.7421 - 6m 5s
batch: 500/1563 - train loss: 3.1935 - test loss: 4.1940 - train acc: 0.7974 - test acc: 0.7426 - 6m 8s
batch: 600/1563 - train loss: 3.1799 - test loss: 4.2597 - train acc: 0.8061 - test acc: 0.7460 - 6m 11s
batch: 700/1563 - train loss: 3.0846 - test loss: 4.1931 - train acc: 0.8080 - test acc: 0.7535 - 6m 14s
batch: 800/1563 - train loss: 3.1846 - test loss: 4.2528 - train acc: 0.8056 - test acc: 0.7463 - 6m 17s
batch: 900/1563 - train loss: 3.1483 - test loss: 4.5165 - train acc: 0.8040 - test acc: 0.7360 - 6m 20s
batch: 1000/1563 - train loss: 3.0799 - test loss: 4.1852 - train acc: 0.8090 - test acc: 0.7510 - 6m 23s
batch: 1100/1563 - train loss: 3.1254 - test loss: 4.0970 - train acc: 0.8012 - test acc: 0.7515 - 6m 26s
batch: 1200/1563 - train loss: 3.1622 - test loss: 4.176

batch: 1300/1563 - train loss: 2.3457 - test loss: 4.3963 - train acc: 0.8572 - test acc: 0.7584 - 9m 33s
batch: 1400/1563 - train loss: 2.2844 - test loss: 4.2774 - train acc: 0.8606 - test acc: 0.7646 - 9m 36s
batch: 1500/1563 - train loss: 2.2703 - test loss: 4.4175 - train acc: 0.8607 - test acc: 0.7568 - 9m 39s
batch: 1563/1563 - train loss: 2.2793 - test loss: 4.4353 - train acc: 0.8581 - test acc: 0.7594 - 9m 41s
GPU memory used: 0.02 GB - max: 3.19 GB - memory reserved: 0.15 GB - max: 3.28 GB
starting epoch: 14/100
batch: 100/1563 - train loss: 1.8180 - test loss: 4.5978 - train acc: 0.8828 - test acc: 0.7656 - 9m 44s
batch: 200/1563 - train loss: 1.6873 - test loss: 4.5465 - train acc: 0.8966 - test acc: 0.7612 - 9m 47s
batch: 300/1563 - train loss: 1.6972 - test loss: 4.6201 - train acc: 0.8948 - test acc: 0.7680 - 9m 50s
batch: 400/1563 - train loss: 1.8010 - test loss: 4.6575 - train acc: 0.8887 - test acc: 0.7587 - 9m 52s
batch: 500/1563 - train loss: 1.7331 - test loss: 4

batch: 600/1563 - train loss: 1.2770 - test loss: 5.1681 - train acc: 0.9242 - test acc: 0.7672 - 13m 2s
batch: 700/1563 - train loss: 1.3690 - test loss: 5.2301 - train acc: 0.9179 - test acc: 0.7614 - 13m 5s
batch: 800/1563 - train loss: 1.5606 - test loss: 5.2335 - train acc: 0.9094 - test acc: 0.7591 - 13m 8s
batch: 900/1563 - train loss: 1.3877 - test loss: 5.5522 - train acc: 0.9126 - test acc: 0.7576 - 13m 11s
batch: 1000/1563 - train loss: 1.4184 - test loss: 5.1074 - train acc: 0.9110 - test acc: 0.7588 - 13m 14s
batch: 1100/1563 - train loss: 1.3933 - test loss: 5.0804 - train acc: 0.9152 - test acc: 0.7557 - 13m 17s
batch: 1200/1563 - train loss: 1.4670 - test loss: 4.9600 - train acc: 0.9041 - test acc: 0.7558 - 13m 20s
batch: 1300/1563 - train loss: 1.3804 - test loss: 5.1829 - train acc: 0.9151 - test acc: 0.7629 - 13m 23s
batch: 1400/1563 - train loss: 1.5440 - test loss: 5.1434 - train acc: 0.9045 - test acc: 0.7563 - 13m 26s
batch: 1500/1563 - train loss: 1.4083 - test

...

batch: 1500/1563 - train loss: 1.1143 - test loss: 5.5031 - train acc: 0.9317 - test acc: 0.7637 - 16m 48s
batch: 1563/1563 - train loss: 1.1590 - test loss: 5.8006 - train acc: 0.9298 - test acc: 0.7559 - 16m 51s
GPU memory used: 0.02 GB - max: 3.19 GB - memory reserved: 0.15 GB - max: 3.28 GB
starting epoch: 23/100
batch: 100/1563 - train loss: 0.9137 - test loss: 5.6559 - train acc: 0.9433 - test acc: 0.7662 - 16m 54s
batch: 200/1563 - train loss: 0.8995 - test loss: 5.8117 - train acc: 0.9471 - test acc: 0.7684 - 16m 57s
batch: 300/1563 - train loss: 0.9794 - test loss: 5.9182 - train acc: 0.9402 - test acc: 0.7682 - 17m 0s
batch: 400/1563 - train loss: 0.9136 - test loss: 5.7554 - train acc: 0.9451 - test acc: 0.7652 - 17m 3s
batch: 500/1563 - train loss: 1.0405 - test loss: 6.0557 - train acc: 0.9383 - test acc: 0.7546 - 17m 6s
batch: 600/1563 - train loss: 0.9960 - test loss: 5.9435 - train acc: 0.9383 - test acc: 0.7596 - 17m 9s
batch: 700/1563 - train loss: 1.1219 - test loss:

batch: 100/1563 - train loss: 13.0313 - test loss: 13.0162 - train acc: 0.0933 - test acc: 0.1534 - 0m 0s
batch: 200/1563 - train loss: 12.9652 - test loss: 12.7591 - train acc: 0.1366 - test acc: 0.1318 - 0m 3s
batch: 300/1563 - train loss: 12.2500 - test loss: 12.0883 - train acc: 0.1882 - test acc: 0.2058 - 0m 6s
batch: 400/1563 - train loss: 11.5819 - test loss: 11.2040 - train acc: 0.2329 - test acc: 0.2798 - 0m 9s
batch: 500/1563 - train loss: 11.2643 - test loss: 11.0108 - train acc: 0.2616 - test acc: 0.2780 - 0m 12s
batch: 600/1563 - train loss: 10.6610 - test loss: 10.7028 - train acc: 0.2897 - test acc: 0.2807 - 0m 14s
batch: 700/1563 - train loss: 10.3659 - test loss: 9.7108 - train acc: 0.3047 - test acc: 0.3588 - 0m 17s
batch: 800/1563 - train loss: 10.3540 - test loss: 9.7547 - train acc: 0.3056 - test acc: 0.3622 - 0m 20s
batch: 900/1563 - train loss: 9.9751 - test loss: 9.3364 - train acc: 0.3472 - test acc: 0.3866 - 0m 23s
batch: 1000/1563 - train loss: 9.7909 - test 

batch: 1100/1563 - train loss: 4.6690 - test loss: 4.7353 - train acc: 0.7141 - test acc: 0.7123 - 3m 28s
batch: 1200/1563 - train loss: 4.6142 - test loss: 4.7428 - train acc: 0.7157 - test acc: 0.7126 - 3m 31s
batch: 1300/1563 - train loss: 4.7549 - test loss: 4.8865 - train acc: 0.7097 - test acc: 0.6959 - 3m 34s
batch: 1400/1563 - train loss: 4.5524 - test loss: 4.7751 - train acc: 0.7238 - test acc: 0.7062 - 3m 37s
batch: 1500/1563 - train loss: 4.5390 - test loss: 4.7660 - train acc: 0.7272 - test acc: 0.7046 - 3m 39s
batch: 1563/1563 - train loss: 4.5902 - test loss: 4.7104 - train acc: 0.7154 - test acc: 0.7079 - 3m 42s
GPU memory used: 0.02 GB - max: 3.19 GB - memory reserved: 0.15 GB - max: 3.28 GB
starting epoch: 6/100
batch: 100/1563 - train loss: 4.2666 - test loss: 4.6818 - train acc: 0.7293 - test acc: 0.7160 - 3m 45s
batch: 200/1563 - train loss: 4.2048 - test loss: 4.5923 - train acc: 0.7390 - test acc: 0.7157 - 3m 48s
batch: 300/1563 - train loss: 4.2294 - test loss: 

batch: 400/1563 - train loss: 2.7070 - test loss: 4.0659 - train acc: 0.8299 - test acc: 0.7659 - 6m 55s
batch: 500/1563 - train loss: 2.5467 - test loss: 4.0414 - train acc: 0.8368 - test acc: 0.7674 - 6m 57s
batch: 600/1563 - train loss: 2.6948 - test loss: 4.1163 - train acc: 0.8362 - test acc: 0.7553 - 7m 0s
batch: 700/1563 - train loss: 2.7855 - test loss: 4.3070 - train acc: 0.8272 - test acc: 0.7476 - 7m 3s
batch: 800/1563 - train loss: 2.7463 - test loss: 4.0455 - train acc: 0.8259 - test acc: 0.7611 - 7m 6s
batch: 900/1563 - train loss: 2.6017 - test loss: 4.0671 - train acc: 0.8431 - test acc: 0.7648 - 7m 9s
batch: 1000/1563 - train loss: 2.7726 - test loss: 4.2776 - train acc: 0.8243 - test acc: 0.7530 - 7m 12s
batch: 1100/1563 - train loss: 2.7824 - test loss: 4.0873 - train acc: 0.8303 - test acc: 0.7604 - 7m 15s
batch: 1200/1563 - train loss: 2.7904 - test loss: 4.3031 - train acc: 0.8368 - test acc: 0.7532 - 7m 17s
batch: 1300/1563 - train loss: 2.7733 - test loss: 4.127

batch: 1400/1563 - train loss: 1.9664 - test loss: 4.4604 - train acc: 0.8800 - test acc: 0.7659 - 10m 27s
batch: 1500/1563 - train loss: 2.0164 - test loss: 4.1827 - train acc: 0.8753 - test acc: 0.7701 - 10m 30s
batch: 1563/1563 - train loss: 1.9503 - test loss: 4.5813 - train acc: 0.8760 - test acc: 0.7657 - 10m 33s
GPU memory used: 0.02 GB - max: 3.19 GB - memory reserved: 0.15 GB - max: 3.28 GB
starting epoch: 15/100
batch: 100/1563 - train loss: 1.2495 - test loss: 4.9297 - train acc: 0.9239 - test acc: 0.7619 - 10m 35s
batch: 200/1563 - train loss: 1.4123 - test loss: 4.8872 - train acc: 0.9189 - test acc: 0.7681 - 10m 39s
batch: 300/1563 - train loss: 1.4141 - test loss: 4.6223 - train acc: 0.9145 - test acc: 0.7711 - 10m 42s
batch: 400/1563 - train loss: 1.4645 - test loss: 4.8270 - train acc: 0.9028 - test acc: 0.7636 - 10m 45s
batch: 500/1563 - train loss: 1.4360 - test loss: 4.8515 - train acc: 0.9101 - test acc: 0.7727 - 10m 47s
